# 1.샘플 데이터 소개

In [80]:
import findspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import expr
from pyspark.sql.functions import mean, stddev
from pyspark.sql.functions import when



spark = SparkSession.builder \
    .master("local")\
    .appName("Spark MiniProject")\
    .config("spark.sql.rep1.eagerEval.enabled", True)\
    .getOrCreate()

In [9]:
G3WinnerDataFrame = spark.read.option("delimiter", ",").csv("./Rank_winner_gold3.csv", encoding="UTF-8", header=True)
G3WinnerDataFrame.show(5)

+-------------+--------------------+----------------+-------------+----+--------+------------+------------+------------+----------+----------+-------+----------------+--------------+----------------+--------------+-----+----------+-----------+-----------+-----+-----+-----+-----+-----+-----+-----+-------+----+-------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-----------+-----------+---------------------------+----------------+------------------+-----------------------------+-----------+-----------------------+-----------+-----------+
|      matchId|               puuid|    summonerName|summonerLevel| win|gameMode|gameDuration|teamPosition|championName|championId|champLevel|assists|firstBloodAssist|firstBloodKill|firstTowerAssist|firstTowerKill|kills|pentaK

## - csv의 파일(gold3 경기에서 승리를 한 유저들의 게임내 정보)

# 2. 데이터 전처리

In [24]:
G3WinnerDataFrame1 = G3WinnerDataFrame.drop("firstBloodAssist","firstTowerAssist","quadraKills","pentaKills","item0","item1","item2","item3","item4","item5","item6","tripleKills","puuid","summonerLevel","win","gameMode","championId","defense","flex","offense","prim1_perk","prim1_var1","prim1_var2","prim1_var3","prim2_perk","prim2_var1","prim2_var2","prim2_var3","prim3_perk","prim3_var1","prim3_var2","prim3_var3","prim4_perk","prim4_var1","prim4_var2","prim4_var3","prim_style","sub1_perk","sub1_var1","sub1_var2","sub1_var3","sub2_perk","sub2_var1","sub2_var2","sub2_var3","sub_style","summoner1Id","summoner2Id","visionWardsBoughtInGame","wardsKilled","wardsPlaced")
G3WinnerDataFrame1.show(5)

+-------------+----------------+------------+------------+------------+----------+-------+--------------+--------------+-----+---------------------------+----------------+------------------+-----------------------------+-----------+
|      matchId|    summonerName|gameDuration|teamPosition|championName|champLevel|assists|firstBloodKill|firstTowerKill|kills|totalDamageDealtToChampions|totalDamageTaken|totalMinionsKilled|totalEnemyJungleMinionsKilled|visionScore|
+-------------+----------------+------------+------------+------------+----------+-------+--------------+--------------+-----+---------------------------+----------------+------------------+-----------------------------+-----------+
|KR_6611804574|       포비 해리|        2111|         TOP|       Riven|        18|      8|         False|         False|    9|                      36424|           44248|               240|                            4|         15|
|KR_6611804574|제일먼저이를닦자|        2111|      JUNGLE|      RekSai|        1

## - 소규모 분석에 필요하지 않거나 번호id로 지금 당장 확인할수 없는 컬럼들 삭제

In [58]:
G3WinnerDataFrame1 = G3WinnerDataFrame1\
    .withColumnRenamed("totalDamageDealtToChampions","DealtDamage")\
    .withColumnRenamed("totalEnemyJungleMinionsKilled","jungleMinion")\
    .withColumnRenamed("totalMinionsKilled","MinionsKilled")
G3WinnerDataFrame1.show(10)

+-------------+----------------+------------+------------+------------+----------+-------+--------------+--------------+-----+-----------+----------------+-------------+------------+-----------+
|      matchId|    summonerName|gameDuration|teamPosition|championName|champLevel|assists|firstBloodKill|firstTowerKill|kills|DealtDamage|totalDamageTaken|MinionsKilled|jungleMinion|visionScore|
+-------------+----------------+------------+------------+------------+----------+-------+--------------+--------------+-----+-----------+----------------+-------------+------------+-----------+
|KR_6611804574|       포비 해리|        2111|         TOP|       Riven|        18|      8|         False|         False|    9|      36424|           44248|          240|           4|         15|
|KR_6611804574|제일먼저이를닦자|        2111|      JUNGLE|      RekSai|        18|     18|          True|         False|   14|      36867|           41418|           35|           2|         41|
|KR_6611804574|  트롤러의플레이|        2111

## - 테이블의 형태를 유지하기 위하여 불필요하게 긴 문자열 길이 수정
## 현재 가져온 데이터에 아군jungle 몹 cs가 나와있지 않아 부득이하게 표현을 못하였음

In [59]:
G3WinnerDataFrame2 = G3WinnerDataFrame1.withColumn("cs", (col("MinionsKilled").cast(IntegerType()) + col("jungleMinion").cast(IntegerType())).cast(StringType()))
G3WinnerDataFrame2.show()

+-------------+----------------+------------+------------+------------+----------+-------+--------------+--------------+-----+-----------+----------------+-------------+------------+-----------+---+
|      matchId|    summonerName|gameDuration|teamPosition|championName|champLevel|assists|firstBloodKill|firstTowerKill|kills|DealtDamage|totalDamageTaken|MinionsKilled|jungleMinion|visionScore| cs|
+-------------+----------------+------------+------------+------------+----------+-------+--------------+--------------+-----+-----------+----------------+-------------+------------+-----------+---+
|KR_6611804574|       포비 해리|        2111|         TOP|       Riven|        18|      8|         False|         False|    9|      36424|           44248|          240|           4|         15|244|
|KR_6611804574|제일먼저이를닦자|        2111|      JUNGLE|      RekSai|        18|     18|          True|         False|   14|      36867|           41418|           35|           2|         41| 37|
|KR_6611804574|  

## - minionkill과 jungleminionkill을 합쳐 cs를 만들어냄 각 컬럼을 정수형으로 합

In [61]:
G3WinnerDataFrame3 = G3WinnerDataFrame2\
    .withColumn("minutes", expr("floor(gameDuration / 60)"))\
    .withColumn("seconds", expr("gameDuration % 60"))
G3WinnerDataFrame4 = G3WinnerDataFrame3.drop("gameDuration")

G3WinnerDataFrame5 = G3WinnerDataFrame4.sort(G3WinnerDataFrame4.minutes.desc())

G3WinnerDataFrame5.show()

+-------------+----------------+------------+------------+----------+-------+--------------+--------------+-----+-----------+----------------+-------------+------------+-----------+---+-------+-------+
|      matchId|    summonerName|teamPosition|championName|champLevel|assists|firstBloodKill|firstTowerKill|kills|DealtDamage|totalDamageTaken|MinionsKilled|jungleMinion|visionScore| cs|minutes|seconds|
+-------------+----------------+------------+------------+----------+-------+--------------+--------------+-----+-----------+----------------+-------------+------------+-----------+---+-------+-------+
|KR_6654070267|    말랑말랑요정|         TOP|        Yone|        18|     15|         False|         False|   12|      50705|           59890|          338|           8|         59|346|     55|   38.0|
|KR_6654070267|          최수완|      JUNGLE|      LeeSin|        18|     29|         False|         False|   24|      53466|          105217|           56|          27|         68| 83|     55|   38.0|

## - gameduration을 분,초 로 나눠서 가장 게임 시간이 긴 순으로 표시

In [62]:
G3WinnerDataFrame6 = G3WinnerDataFrame5.select(G3WinnerDataFrame5.teamPosition,
                                               G3WinnerDataFrame5.jungleMinion.cast("int"))

G3WinnerDataFrame6.show()

+------------+------------+
|teamPosition|jungleMinion|
+------------+------------+
|         TOP|           8|
|      JUNGLE|          27|
|      MIDDLE|           0|
|      BOTTOM|           8|
|     UTILITY|           0|
|         TOP|          24|
|      JUNGLE|          10|
|      MIDDLE|           5|
|      BOTTOM|          14|
|     UTILITY|           0|
|         TOP|           8|
|      JUNGLE|          30|
|      MIDDLE|          25|
|      BOTTOM|           2|
|     UTILITY|           1|
|     UTILITY|          14|
|         TOP|           0|
|      JUNGLE|          59|
|      MIDDLE|           3|
|      BOTTOM|           1|
+------------+------------+
only showing top 20 rows



In [63]:
G3WinnerDataFrame6.describe().show()

+-------+------------+-----------------+
|summary|teamPosition|     jungleMinion|
+-------+------------+-----------------+
|  count|       19693|            20566|
|   mean|        null|4.130992900904405|
| stddev|        null|7.836243713807128|
|    min|      BOTTOM|                0|
|    max|     UTILITY|               88|
+-------+------------+-----------------+



## - 라인과 적정글 미니언을 카운트하여 적 정글에 얼마나 많은 개입을 했는지 요약정보 출력
## - 적정글 개입 평균, 표준편차, 최솟값, 최대값 확인

In [86]:
G3WinnerDataFrame7 = G3WinnerDataFrame6.select(G3WinnerDataFrame6.teamPosition.alias("라인"), G3WinnerDataFrame6.jungleMinion.alias("상대정글견제"),
                          when(G3WinnerDataFrame6.jungleMinion > 13, "잘먹음").
                          when((G3WinnerDataFrame6.jungleMinion <13) & (G3WinnerDataFrame6.jungleMinion > 4), "먹음").
                          otherwise("잘안먹음").alias("상대 정글 견제 능력"))

G3WinnerDataFrame7.show()

+-------+------------+-------------------+
|   라인|상대정글견제|상대 정글 견제 능력|
+-------+------------+-------------------+
|    TOP|           8|               먹음|
| JUNGLE|          27|             잘먹음|
| MIDDLE|           0|           잘안먹음|
| BOTTOM|           8|               먹음|
|UTILITY|           0|           잘안먹음|
|    TOP|          24|             잘먹음|
| JUNGLE|          10|               먹음|
| MIDDLE|           5|               먹음|
| BOTTOM|          14|             잘먹음|
|UTILITY|           0|           잘안먹음|
|    TOP|           8|               먹음|
| JUNGLE|          30|             잘먹음|
| MIDDLE|          25|             잘먹음|
| BOTTOM|           2|           잘안먹음|
|UTILITY|           1|           잘안먹음|
|    TOP|           0|           잘안먹음|
| JUNGLE|          59|             잘먹음|
| MIDDLE|           3|           잘안먹음|
| BOTTOM|           1|           잘안먹음|
|UTILITY|          14|             잘먹음|
+-------+------------+-------------------+
only showing top 20 rows



## - 상대정글 견제를 상대정글몹 먹은 양에 따라 구분하여 능력이라는 새로운 필드를 만들어 출력

In [87]:
G3WinnerDataFrame7.sort(G3WinnerDataFrame7.상대정글견제.desc()).show()

+------+------------+-------------------+
|  라인|상대정글견제|상대 정글 견제 능력|
+------+------------+-------------------+
|JUNGLE|          88|             잘먹음|
|JUNGLE|          85|             잘먹음|
|JUNGLE|          83|             잘먹음|
|JUNGLE|          73|             잘먹음|
|JUNGLE|          73|             잘먹음|
|JUNGLE|          70|             잘먹음|
|JUNGLE|          66|             잘먹음|
|JUNGLE|          66|             잘먹음|
|JUNGLE|          65|             잘먹음|
|JUNGLE|          65|             잘먹음|
|JUNGLE|          61|             잘먹음|
|JUNGLE|          61|             잘먹음|
|JUNGLE|          61|             잘먹음|
|JUNGLE|          60|             잘먹음|
|JUNGLE|          60|             잘먹음|
|JUNGLE|          59|             잘먹음|
|JUNGLE|          59|             잘먹음|
|JUNGLE|          59|             잘먹음|
|JUNGLE|          59|             잘먹음|
|JUNGLE|          58|             잘먹음|
+------+------------+-------------------+
only showing top 20 rows



## - 내림차순으로 정렬 5000개의 데이터중 상위 20명은 모두 정글러임
##   아무래도 라이너보다 발이 풀려있다보니 상대 정글에 개입할 가능성이 크고
##   같은 정글이다보니 이러한 결과가 나온것으로 확인됨 